In [ ]:
import pandas as pd
from google.cloud import bigquery
import os
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from dotenv import load_dotenv

In [ ]:
import google.auth

# Verifica a conta de serviço atual
credentials, project = google.auth.default()
print(f"Usando a conta de serviço: {credentials.service_account_email}")
print(f"Projeto: {project}")

In [55]:
# Inicializar o cliente BigQuery usando as credenciais do arquivo .env
client = bigquery.Client()

# Carregar as variáveis de ambiente do arquivo .env
load_dotenv()

# Obter o caminho do arquivo CSV a partir das variáveis de ambiente
csv_path = os.getenv('CSV_FILE_PATH')

# Verificar se a variável está definida corretamente
if not csv_path:
    raise ValueError("CSV_FILE_PATH não está definida no arquivo .env")

# Ler o arquivo CSV usando o caminho fornecido pela variável de ambiente
df_apartamentos = pd.read_csv(csv_path)

# Limpar e verificar dados (strip para colunas e dados)
df_apartamentos.columns = df_apartamentos.columns.str.strip()
df_apartamentos = df_apartamentos.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Renomear as colunas do CSV para corresponder às colunas do BigQuery
df_apartamentos = df_apartamentos.rename(columns={
    "Código": "Codigo",
    "Nome/Endereço": "Nome_Endereco",
    "Bairro": "Bairro",
    "Valor Total Mensal (R$)": "Valor_Total_Mensal",
    "Preço do Aluguel (R$)": "Preco_Aluguel",
    "Condomínio (R$)": "Condominio",
    "IPTU (R$)": "IPTU",
    "Quartos": "Quartos",
    "Banheiros": "Banheiros",
    "Vagas": "Vagas",
    "Área (m²)": "Area",
    "Facilidades": "Facilidades",
    "Imobiliária": "Imobiliaria",
    "Fonte (site)": "Fonte"
})

In [79]:
# Verificar se o apartamento já existe no BigQuery pela coluna 'Fonte' na tabela 'apartamentos'
query_apartamentos = """
SELECT Fonte FROM `rafael-data.ranking_apartamentos_flask.apartamentos`
"""
existing_apartments = client.query(query_apartamentos).to_dataframe()

# Filtrar os apartamentos que não estão na tabela 'lista_de_imoveis_pontuados'
new_apartments = df_apartamentos[~df_apartamentos['Fonte'].isin(existing_apartments['Fonte'])]

# Verificar se os registros da tabela 'lista_de_imoveis_pontuados' já foram pontuados
query_lista_pontuados = """
SELECT Fonte FROM `rafael-data.ranking_apartamentos_flask.lista_de_imoveis_pontuados`
"""
existing_pontuados = client.query(query_lista_pontuados).to_dataframe()

# Filtrar os apartamentos que ainda não foram pontuados na tabela 'lista_de_imoveis_pontuados'
new_apartments = new_apartments[~new_apartments['Fonte'].isin(existing_pontuados['Fonte'])]

# Verifique se há novos apartamentos para processar
if not new_apartments.empty:
    # Carregar os novos apartamentos na tabela 'apartamentos' do BigQuery
    job = client.load_table_from_dataframe(new_apartments, 'rafael-data.ranking_apartamentos_flask.apartamentos')
    job.result()  # Espera o job de carregamento terminar

else:
    print("Nenhum novo apartamento para processar.")


Nenhum novo apartamento para processar.


In [80]:
# Query única para carregar todos os registros da tabela 'apartamentos', excluindo os que já estão na tabela 'lista_de_imoveis_pontuados'
query_apartamentos_filtrados = """
SELECT *
FROM `rafael-data.ranking_apartamentos_flask.apartamentos`
WHERE Fonte NOT IN (
    SELECT Fonte FROM `rafael-data.ranking_apartamentos_flask.lista_de_imoveis_pontuados`
)
"""

# Executar a query e carregar os resultados em um DataFrame
new_apartments = client.query(query_apartamentos_filtrados).to_dataframe()

# Exibir o DataFrame resultante
new_apartments

    Codigo                                      Nome_Endereco  \
0       45        Rua Joaquim Távora, Vila Mariana, São Paulo   
1       46  Rua Napoleão de Barros, Vila Clementino, São P...   
2       47           Rua Loefgren, Vila Clementino, São Paulo   
3       42                Rua Leandro Dupret, Vila Clementino   
4       44                  Rua França Pinto, Vila Clementino   
5       37            Rua Maestro Elias Lobo, Jardim Paulista   
6       36                     Rua Alves Guimarães, Pinheiros   
7       41                              Bela Vista, São Paulo   
8       39                  Rua França Pinto, Vila Clementino   
9       40                     Rua França Pinto, Vila Mariana   
10      43                                Ipiranga, São Paulo   
11      38                           Rua Estela, Vila Mariana   

             Bairro  Valor_Total_Mensal  Preco_Aluguel  Condominio   IPTU  \
0      Vila Mariana              4406.0         3900.0       300.0  206.0   


In [81]:
# Configurações de colunas com base no BigQuery
nome_coluna_codigo = 'Codigo'
nome_coluna_nome_endereco = 'Nome_Endereco'
nome_coluna_bairro = 'Bairro'
nome_coluna_valor_total_mensal = 'Valor_Total_Mensal'
nome_coluna_preco_aluguel = 'Preco_Aluguel'
nome_coluna_condominio = 'Condominio'
nome_coluna_iptu = 'IPTU'
nome_coluna_quartos = 'Quartos'
nome_coluna_banheiros = 'Banheiros'
nome_coluna_vagas = 'Vagas'
nome_coluna_area = 'Area'
nome_coluna_facilidades = 'Facilidades'
nome_coluna_imobiliaria = 'Imobiliaria'
nome_coluna_fonte = 'Fonte'  # Nome da coluna no BigQuery que armazena o link do site do apartamento
nome_coluna_latlong = 'LatLong'
nome_coluna_distancia_interlagos = 'Distancia_Interlagos'

In [82]:
geolocator = Nominatim(user_agent="geoapi")

# Função para buscar coordenadas de um bairro
def buscar_coordenadas(bairro):
    location = geolocator.geocode(bairro + ", São Paulo, Brazil")
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

# Aplicando coordenadas no DataFrame
new_apartments[nome_coluna_latlong] = new_apartments[nome_coluna_bairro].apply(buscar_coordenadas)

In [83]:
# Função para calcular a distância entre o apartamento e o trabalho (Interlagos)
def calcular_distancia_robusta(linha):
    coords_trabalho = (-23.6842, -46.6802)
    localizacao = linha[nome_coluna_latlong]

    if pd.notnull(localizacao):
        return geodesic(localizacao, coords_trabalho).km
    else:
        return None

new_apartments[nome_coluna_distancia_interlagos] = new_apartments.apply(calcular_distancia_robusta, axis=1)

new_apartments

,Codigo,Nome_Endereco,Bairro,Valor_Total_Mensal,Preco_Aluguel,Condominio,IPTU,Quartos,Banheiros,Vagas,Area,Facilidades,Imobiliaria,Fonte,LatLong,Distancia_Interlagos
0,45,"Rua Joaquim Távora, Vila Mariana, São Paulo",Vila Mariana,4406.0,3900.0,300.0,206.0,2,2,0,90.0,"Banheira de hidromassagem, armários na cozinha...",QuintoAndar,https://www.quintoandar.com.br/imovel/894609497,"(-23.5837, -46.632740824206934)",12.138484
1,46,"Rua Napoleão de Barros, Vila Clementino, São P...",Vila Clementino,4867.0,3800.0,939.0,128.0,2,2,0,75.0,"Ar condicionado, armários embutidos, cozinha p...",QuintoAndar,https://www.quintoandar.com.br/imovel/893244748,"(-23.5980058, -46.6434179)",10.257482
2,47,"Rua Loefgren, Vila Clementino, São Paulo",Vila Clementino,4384.0,3900.0,352.0,132.0,2,3,1,55.0,"Armários embutidos, salão de festas",QuintoAndar,https://www.quintoandar.com.br/imovel/893903049,"(-23.5980058, -46.6434179)",10.257482
3,42,"Rua Leandro Dupret, Vila Clementino",Vila Clementino,4400.0,3600.0,675.0,125.0,2,1,1,70.0,"Elevador, portaria 24h, próximo ao metrô",MMG Imóveis,https://www.mmgimoveis.com.br/imovel/apartamen...,"(-23.5980058, -46.6434179)",10.257482
4,44,"Rua França Pinto, Vila Clementino",Vila Clementino,4267.0,3250.0,985.0,32.0,2,1,1,80.0,"Elevador, portaria 24h, salão de festas, garag...",MMG Imóveis,https://www.mmgimoveis.com.br/imovel/apartamen...,"(-23.5980058, -46.6434179)",10.257482
5,37,"Rua Maestro Elias Lobo, Jardim Paulista",Jardim Paulista,4107.0,3512.0,560.0,35.0,2,2,0,70.0,"Elevador, portaria 24h, aceita pets, armários ...",QuintoAndar,https://www.quintoandar.com.br/imovel/89410335...,"(-23.5652185, -46.6601846)",13.334885
6,36,"Rua Alves Guimarães, Pinheiros",Pinheiros,4981.0,3500.0,1481.0,0.0,2,2,1,86.0,"Próximo ao metrô, elevador, portaria 24h, salã...",QuintoAndar,https://www.quintoandar.com.br/imovel/89345282...,"(-23.567249, -46.7019515)",13.141459
7,41,"Bela Vista, São Paulo",Bela Vista,5255.0,3950.0,1170.0,135.0,2,2,1,83.0,"Portaria 24h, salão de festas",MMG Imóveis,https://www.mmgimoveis.com.br/imovel/apartamen...,"(-23.5601219, -46.6500338)",14.082569
8,39,"Rua França Pinto, Vila Clementino",Vila Clementino,5820.0,4000.0,1300.0,520.0,3,2,2,84.0,"Academia, piscina, churrasqueira, salão de fes...",MMG Imóveis,https://www.mmgimoveis.com.br/imovel/apartamen...,"(-23.5980058, -46.6434179)",10.257482
9,40,"Rua França Pinto, Vila Mariana",Vila Mariana,5725.0,4000.0,1375.0,350.0,2,2,2,65.0,"Piscina, academia, salão de festas, playground",MMG Imóveis,https://www.mmgimoveis.com.br/imovel/apartamen...,"(-23.5837, -46.632740824206934)",12.138484


In [84]:
# Função para calcular pontuações no DataFrame
def calcular_pontuacoes(df):
    if df.empty:
        return df  # Retornar o DataFrame vazio se não houver dados
    df['Pontuacao_Total'] = (df['Pontos_Preco'] + df['Pontos_Distancia'] +
                             df['Pontos_Quartos'] + df['Pontos_Banheiros'] +
                             df['Pontos_Vagas'] + df['Pontos_Area'] +
                             df['Pontos_Facilidades'] + df['Pontos_Fonte'])
    return df

# Função para calcular a pontuação atualizada para cada apartamento
def calcular_pontuacao_atualizada(apartamento):
    # Critério 1: Preço Total (30%)
    if apartamento[nome_coluna_valor_total_mensal] < 4500:
        pontos_preco = 10 + ((4500 - apartamento[nome_coluna_valor_total_mensal]) // 100) * 10
    elif apartamento[nome_coluna_valor_total_mensal] > 4500:
        pontos_preco = -((apartamento[nome_coluna_valor_total_mensal] - 4500) // 200) * 10
    else:
        pontos_preco = 0

    # Critério 2: Distância até Interlagos (20%)
    if apartamento[nome_coluna_distancia_interlagos] < 15:
        pontos_distancia = 5 + ((15 - apartamento[nome_coluna_distancia_interlagos]) // 2) * 5
    else:
        pontos_distancia = -(apartamento[nome_coluna_distancia_interlagos] - 15) // 2 * 5

    # Critério 3: Número de Quartos (parte de 10%)
    pontos_quartos = (apartamento[nome_coluna_quartos] - 2) * 5 if apartamento[nome_coluna_quartos] > 2 else 0

    # Critério 4: Banheiros (parte de 10%)
    pontos_banheiros = (apartamento[nome_coluna_banheiros] - 2) * 3 if apartamento[nome_coluna_banheiros] > 2 else 0

    # Critério 5: Vagas (parte de 8%)
    pontos_vagas = apartamento[nome_coluna_vagas] * 2 if apartamento[nome_coluna_vagas] > 0 else 0

    # Critério 6: Área do Apartamento (10%)
    if apartamento[nome_coluna_area] >= 75:
        pontos_area = 1 + ((apartamento[nome_coluna_area] - 75) // 10) * 2
    else:
        pontos_area = -1 - ((75 - apartamento[nome_coluna_area]) // 10) * 2

    # Critério 7: Facilidades (5%)
    num_facilidades = len(apartamento[nome_coluna_facilidades].split(", "))
    pontos_facilidades = 2 * num_facilidades

    # Critério 8: Fonte (7%)
    apartamento[nome_coluna_imobiliaria] = apartamento[nome_coluna_imobiliaria].lower()
    pontos_fonte = 15 if "quintoandar" in apartamento[nome_coluna_imobiliaria].lower() else 0

    # Calculando a pontuação total
    pontuacao_total = (
        (pontos_preco * 0.3) +
        (pontos_distancia * 0.2) +
        (pontos_quartos * 0.1) +
        (pontos_banheiros * 0.1) +
        (pontos_vagas * 0.08) +
        (pontos_area * 0.1) +
        (pontos_facilidades * 0.05) +
        (pontos_fonte * 0.07)
    )

    return {
        "Pontos_Preco": pontos_preco,
        "Pontos_Distancia": pontos_distancia,
        "Pontos_Quartos": pontos_quartos,
        "Pontos_Banheiros": pontos_banheiros,
        "Pontos_Vagas": pontos_vagas,
        "Pontos_Area": pontos_area,
        "Pontos_Facilidades": pontos_facilidades,
        "Pontos_Fonte": pontos_fonte,
        "Pontuacao_Total": pontuacao_total
    }

# Aplicar a função calcular_pontuacao_atualizada no DataFrame
def aplicar_pontuacao(df):
    if df.empty:
        return df  # Retornar o DataFrame vazio se não houver dados
    # Aqui, aplicamos a função e criamos um novo DataFrame com as pontuações
    pontuacoes = df.apply(calcular_pontuacao_atualizada, axis=1).apply(pd.Series)
    return pd.concat([df, pontuacoes], axis=1)  # Concatenar o DataFrame original com as pontuações

# Verificar se o DataFrame new_apartments está vazio
if new_apartments.empty:
    print("Nenhum apartamento novo para processar.")
else:
    # Aplicar as pontuações ao DataFrame
    df_apartamentos = aplicar_pontuacao(new_apartments)

    # Agora, calcular a pontuação total com a função calcular_pontuacoes
    df_pontuado = calcular_pontuacoes(df_apartamentos)

    # Exibir o DataFrame final (caso necessário)
    display(df_pontuado)

,Codigo,Nome_Endereco,Bairro,Valor_Total_Mensal,Preco_Aluguel,Condominio,IPTU,Quartos,Banheiros,Vagas,...,Distancia_Interlagos,Pontos_Preco,Pontos_Distancia,Pontos_Quartos,Pontos_Banheiros,Pontos_Vagas,Pontos_Area,Pontos_Facilidades,Pontos_Fonte,Pontuacao_Total
0,45,"Rua Joaquim Távora, Vila Mariana, São Paulo",Vila Mariana,4406.0,3900.0,300.0,206.0,2,2,0,...,12.138484,10.0,10.0,0.0,0.0,0.0,3.0,4.0,15.0,42.0
1,46,"Rua Napoleão de Barros, Vila Clementino, São P...",Vila Clementino,4867.0,3800.0,939.0,128.0,2,2,0,...,10.257482,-10.0,15.0,0.0,0.0,0.0,1.0,6.0,15.0,27.0
2,47,"Rua Loefgren, Vila Clementino, São Paulo",Vila Clementino,4384.0,3900.0,352.0,132.0,2,3,1,...,10.257482,20.0,15.0,0.0,3.0,2.0,-5.0,4.0,15.0,54.0
3,42,"Rua Leandro Dupret, Vila Clementino",Vila Clementino,4400.0,3600.0,675.0,125.0,2,1,1,...,10.257482,20.0,15.0,0.0,0.0,2.0,-1.0,6.0,0.0,42.0
4,44,"Rua França Pinto, Vila Clementino",Vila Clementino,4267.0,3250.0,985.0,32.0,2,1,1,...,10.257482,30.0,15.0,0.0,0.0,2.0,1.0,10.0,0.0,58.0
5,37,"Rua Maestro Elias Lobo, Jardim Paulista",Jardim Paulista,4107.0,3512.0,560.0,35.0,2,2,0,...,13.334885,40.0,5.0,0.0,0.0,0.0,-1.0,10.0,15.0,69.0
6,36,"Rua Alves Guimarães, Pinheiros",Pinheiros,4981.0,3500.0,1481.0,0.0,2,2,1,...,13.141459,-20.0,5.0,0.0,0.0,2.0,3.0,10.0,15.0,15.0
7,41,"Bela Vista, São Paulo",Bela Vista,5255.0,3950.0,1170.0,135.0,2,2,1,...,14.082569,-30.0,5.0,0.0,0.0,2.0,1.0,4.0,0.0,-18.0
8,39,"Rua França Pinto, Vila Clementino",Vila Clementino,5820.0,4000.0,1300.0,520.0,3,2,2,...,10.257482,-60.0,15.0,5.0,0.0,4.0,1.0,12.0,0.0,-23.0
9,40,"Rua França Pinto, Vila Mariana",Vila Mariana,5725.0,4000.0,1375.0,350.0,2,2,2,...,12.138484,-60.0,10.0,0.0,0.0,4.0,-3.0,8.0,0.0,-41.0


In [85]:
# Variável para identificar a tabela 'lista_de_imoveis_pontuados'
lista_de_imoveis_table_id = 'rafael-data.ranking_apartamentos_flask.lista_de_imoveis_pontuados'

# Preencher NaN e converter tipos
df_pontuado.fillna({
    'Pontuacao_Total': 0,
    'Pontos_Preco': 0,
    'Pontos_Distancia': 0,
    'Pontos_Quartos': 0,
    'Pontos_Banheiros': 0,
    'Pontos_Vagas': 0,
    'Pontos_Area': 0,
    'Pontos_Facilidades': 0,
    'Pontos_Fonte': 0
}, inplace=True)

df_pontuado = df_pontuado.astype({
    'Pontuacao_Total': 'float',
    'Pontos_Preco': 'float',
    'Pontos_Distancia': 'float',
    'Pontos_Quartos': 'float',
    'Pontos_Banheiros': 'float',
    'Pontos_Vagas': 'float',
    'Pontos_Area': 'float',
    'Pontos_Facilidades': 'float',
    'Pontos_Fonte': 'float',
    'Codigo': 'int',
    'Quartos': 'int',
    'Banheiros': 'int',
    'Vagas': 'int',
    'Area': 'float'
})

# Variável para contar o número de linhas inseridas
linhas_inseridas = 0

# Loop para inserir os novos apartamentos na tabela 'lista_de_imoveis_pontuados'
for index, row in df_pontuado.iterrows():
    insert_query = f"""
    INSERT INTO `{lista_de_imoveis_table_id}` (
        Pontuacao_Total, Pontos_Preco, Pontos_Distancia, Pontos_Quartos, 
        Pontos_Banheiros, Pontos_Vagas, Pontos_Area, Pontos_Facilidades, 
        Pontos_Fonte, Codigo, Nome_Endereco, Bairro, Valor_Total_Mensal, 
        Preco_Aluguel, Condominio, IPTU, Quartos, Banheiros, Vagas, Area, 
        Facilidades, Imobiliaria, Fonte, LatLong, Distancia_Interlagos
    )
    VALUES (
        {row['Pontuacao_Total']}, {row['Pontos_Preco']}, {row['Pontos_Distancia']}, 
        {row['Pontos_Quartos']}, {row['Pontos_Banheiros']}, {row['Pontos_Vagas']}, 
        {row['Pontos_Area']}, {row['Pontos_Facilidades']}, {row['Pontos_Fonte']}, 
        {row['Codigo']}, '{row['Nome_Endereco']}', '{row['Bairro']}', 
        {row['Valor_Total_Mensal']}, {row['Preco_Aluguel']}, {row['Condominio']}, 
        {row['IPTU']}, {row['Quartos']}, {row['Banheiros']}, {row['Vagas']}, 
        {row['Area']}, '{row['Facilidades']}', '{row['Imobiliaria']}', 
        '{row['Fonte']}', '{row['LatLong']}', {row['Distancia_Interlagos']}
    )
    """
    client.query(insert_query)
    linhas_inseridas += 1

# Imprimir o total de linhas inseridas
print(f"Total de {linhas_inseridas} apartamentos inseridos na tabela 'lista_de_imoveis_pontuados'.")

Total de 12 apartamentos inseridos na tabela 'lista_de_imoveis_pontuados'.


In [86]:
# Define o nome da tabela que contém os apartamentos desconsiderados
tabela_apartamentos_desconsiderados = 'rafael-data.ranking_apartamentos_flask.ranking_apartamentos_desconsiderados'
tabela_lista_de_imoveis_pontuados = 'rafael-data.ranking_apartamentos_flask.lista_de_imoveis_pontuados'

lista_de_imoveis_pontuados_query = f"""
SELECT DISTINCT * FROM `{tabela_lista_de_imoveis_pontuados}`
"""  

lista_de_imoveis_pontuados = client.query(lista_de_imoveis_pontuados_query).result().to_dataframe()

# Cria a query SQL para selecionar as fontes dos apartamentos desconsiderados
# A função DISTINCT garante que as fontes duplicadas não sejam retornadas
select_desconsiderados = f"""
SELECT DISTINCT Fonte FROM `{tabela_apartamentos_desconsiderados}`
"""

# Executa a query no BigQuery e converte o resultado para um DataFrame
imoveis_desconsiderados = client.query(select_desconsiderados).result().to_dataframe()

# Extrai a coluna 'Fonte' do DataFrame e converte para uma lista
# Essa lista conterá todas as fontes dos apartamentos desconsiderados
fontes_desconsideradas = imoveis_desconsiderados['Fonte'].tolist()

# Atualiza o DataFrame df_pontuado, removendo as linhas cujas fontes estão na lista de fontes desconsideradas
# A função 'isin()' verifica se a coluna 'Fonte' está na lista, e o operador '~' inverte a condição, ou seja, mantém apenas os que não estão na lista

df_pontuado = lista_de_imoveis_pontuados[~lista_de_imoveis_pontuados['Fonte'].isin(fontes_desconsideradas)]

df_pontuado

,Pontuacao_Total,Pontos_Preco,Pontos_Distancia,Pontos_Quartos,Pontos_Banheiros,Pontos_Vagas,Pontos_Area,Pontos_Facilidades,Pontos_Fonte,Codigo,...,Quartos,Banheiros,Vagas,Area,Facilidades,Imobiliaria,Fonte,LatLong,Distancia_Interlagos,row_num
6,-660.0,10.0,-690.0,0.0,0.0,2.0,1.0,2.0,15.0,51,...,2,2,1,75.0,Uma vaga na garagem,QuintoAndar,https://www.quintoandar.com.br/imovel/894066064,"(-22.3163987, -49.0888056)",289.683282,1
9,62.0,30.0,5.0,5.0,0.0,2.0,1.0,4.0,15.0,12,...,3,2,1,78.0,"Portaria 24h, salão de festas",QuintoAndar,https://www.quintoandar.com.br/imovel/894590276,"(-23.5601219, -46.6500338)",14.082569,1
12,75.0,30.0,10.0,5.0,0.0,0.0,9.0,6.0,15.0,30,...,3,2,0,119.0,"Elevador, portaria 24h, aceita pets",QuintoAndar,https://www.quintoandar.com.br/imovel/894614690,"(-23.5837, -46.632740824206934)",12.138484,1
14,31.0,-10.0,10.0,5.0,0.0,4.0,5.0,2.0,15.0,50,...,3,2,2,95.0,Duas vagas na garagem,QuintoAndar,https://www.quintoandar.com.br/imovel/893391742,"(-23.5837, -46.632740824206934)",12.138484,1
15,26.0,-10.0,10.0,0.0,0.0,0.0,5.0,6.0,15.0,48,...,2,2,0,104.0,"Armários embutidos, portaria 24h, elevador",QuintoAndar,https://www.quintoandar.com.br/imovel/89297858...,"(-23.5837, -46.632740824206934)",12.138484,1
21,12.0,-30.0,10.0,0.0,0.0,2.0,1.0,14.0,15.0,25,...,2,2,1,80.0,"Playground, Quadra esportiva, Academia, Salão ...",quintoandar,https://www.quintoandar.com.br/imovel/894561348,"(-23.5837, -46.632740824206934)",12.138484,1
23,27.0,-10.0,15.0,5.0,0.0,4.0,1.0,12.0,0.0,34,...,3,2,2,80.0,"Playground, salão de festas, churrasqueira, bi...",MMG Imóveis,https://www.mmgimoveis.com.br/imovel/apartamen...,"(-23.608307, -46.6323198)",9.721879,1
28,42.0,10.0,10.0,0.0,0.0,0.0,3.0,4.0,15.0,45,...,2,2,0,90.0,"Banheira de hidromassagem, armários na cozinha...",QuintoAndar,https://www.quintoandar.com.br/imovel/894609497,"(-23.5837, -46.632740824206934)",12.138484,<NA>
30,27.0,-10.0,15.0,0.0,0.0,0.0,1.0,6.0,15.0,46,...,2,2,0,75.0,"Ar condicionado, armários embutidos, cozinha p...",QuintoAndar,https://www.quintoandar.com.br/imovel/893244748,"(-23.5980058, -46.6434179)",10.257482,<NA>
37,33.0,10.0,10.0,0.0,0.0,4.0,-3.0,12.0,0.0,43,...,2,2,2,60.0,"Piscina, academia, playground, churrasqueira, ...",MMG Imóveis,https://www.mmgimoveis.com.br/imovel/apartamen...,"(-23.58927255, -46.60616206667652)",12.946194,<NA>


In [87]:
# Definir o nome da tabela 'ranking_apartamentos'
table_id = 'rafael-data.ranking_apartamentos_flask.ranking_apartamentos'
# temp_table_id = 'rafael-data.ranking_apartamentos_flask.ranking_apartamentos_temp'

# # Criar uma nova tabela temporária sem os apartamentos desconsiderados
# query_create_temp_table = f"""
# CREATE OR REPLACE TABLE `{temp_table_id}` AS
# SELECT Codigo, Nome_Endereco, Bairro, Valor_Total_Mensal, Fonte, Pontuacao_Total, rank
# FROM `{table_id}`
# """

# # Executar a criação da tabela temporária no BigQuery
# client.query(query_create_temp_table).result()
# print("Tabela temporária criada com sucesso.")

# Apagar a tabela 'ranking_apartamentos' se ela já existir para recriá-la
client.delete_table(table_id, not_found_ok=True)
print(f"Tabela {table_id} excluída, se existia.")

# Definir o schema da tabela sem a coluna 'rank' (será recalculada depois)
schema = [
    bigquery.SchemaField("Codigo", "INT64"),
    bigquery.SchemaField("Nome_Endereco", "STRING"),
    bigquery.SchemaField("Bairro", "STRING"),
    bigquery.SchemaField("Valor_Total_Mensal", "FLOAT64"),
    bigquery.SchemaField("Fonte", "STRING"),
    bigquery.SchemaField("Pontuacao_Total", "FLOAT64"),
    bigquery.SchemaField("rank", "INT64")  # Incluir a coluna 'rank'
]

# Recriar a tabela com o novo schema
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)
print(f"Tabela {table_id} recriada.")

# Ler os dados da tabela temporária para um DataFrame (excluindo a coluna 'rank')
# query_temp_table = f"""
# SELECT Codigo, Nome_Endereco, Bairro, Valor_Total_Mensal, Fonte, Pontuacao_Total 
# FROM `{temp_table_id}`
# """
# df_temp = client.query(query_temp_table).result().to_dataframe()

# # Selecionar somente as colunas relevantes do df_pontuado (mesmo nome das colunas)
# df_pontuado_clean = df_pontuado[['Codigo', 'Nome_Endereco', 'Bairro', 'Valor_Total_Mensal', 'Fonte', 'Pontuacao_Total']]

# # Combinar os dados da tabela temporária com o DataFrame df_pontuado
# df_combined = pd.concat([df_temp, df_pontuado_clean])

# Ordenar o DataFrame combinado por 'Pontuacao_Total' de forma decrescente
df_pontuado = df_pontuado.sort_values(by='Pontuacao_Total', ascending=False)

# Calcular e adicionar a coluna de ranking ao DataFrame combinado
df_pontuado['rank'] = df_pontuado['Pontuacao_Total'].rank(method='first', ascending=False).astype(int)

# Verificar o conteúdo de df_combined
# print("Conteúdo de df_combined:")
# print(df_combined)

# Inserir os dados combinados na tabela recriada
rows_to_insert = [
    {
        "Codigo": row['Codigo'],
        "Nome_Endereco": row['Nome_Endereco'],
        "Bairro": row['Bairro'],
        "Valor_Total_Mensal": row['Valor_Total_Mensal'],
        "Fonte": row['Fonte'],
        "Pontuacao_Total": row['Pontuacao_Total'],
        'rank': row['rank']
    }
    for index, row in df_pontuado.iterrows()
]

# Verificar o conteúdo de rows_to_insert antes da inserção
print("Conteúdo de rows_to_insert:")
for row in rows_to_insert:
    print(row)

# Inserir os dados no BigQuery
errors = client.insert_rows_json(table_id, rows_to_insert)

# Verificar se houve erros
if errors == []:
    print(f"Total de apartamentos inseridos: {len(rows_to_insert)}")
    print("Dados inseridos com sucesso!")
else:
    print(f"Erros encontrados ao inserir os dados: {errors}")

Tabela rafael-data.ranking_apartamentos_flask.ranking_apartamentos excluída, se existia.
Tabela rafael-data.ranking_apartamentos_flask.ranking_apartamentos recriada.
Conteúdo de rows_to_insert:
{'Codigo': 30, 'Nome_Endereco': 'Apartamento - Rua José Antônio Coelho', 'Bairro': 'Vila Mariana', 'Valor_Total_Mensal': 4207.0, 'Fonte': 'https://www.quintoandar.com.br/imovel/894614690', 'Pontuacao_Total': 75.0, 'rank': 1}
{'Codigo': 12, 'Nome_Endereco': 'Apartamento no bairro Bela Vista, São Paulo', 'Bairro': 'Bela Vista', 'Valor_Total_Mensal': 4250.0, 'Fonte': 'https://www.quintoandar.com.br/imovel/894590276', 'Pontuacao_Total': 62.0, 'rank': 2}
{'Codigo': 45, 'Nome_Endereco': 'Rua Joaquim Távora, Vila Mariana, São Paulo', 'Bairro': 'Vila Mariana', 'Valor_Total_Mensal': 4406.0, 'Fonte': 'https://www.quintoandar.com.br/imovel/894609497', 'Pontuacao_Total': 42.0, 'rank': 3}
{'Codigo': 43, 'Nome_Endereco': 'Ipiranga, São Paulo', 'Bairro': 'Ipiranga', 'Valor_Total_Mensal': 4448.0, 'Fonte': 'http